In [ ]:
'''
This python code reprojects the geospatial data used in the model for wildfire susceptibility prediction. 
This code is written by Saurav Dey Shuvo (saurav.met@gmail.com). 
'''

In [ ]:
# Libraries 
import os
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import fiona
from pyproj import CRS
import netCDF4
import shutil
import pygrib

# Target CRS (EPSG: 32617 similar to the circuit shapefile)
TARGET_CRS = 'EPSG:32617'

# File paths
data_folders = {
    "HRRR": "D:/2Courses/8902/data/hrrr/",
    "LandCover": "D:/2Courses/8902/data_model_build/LandCover/land_cover1.nc",
    "SlopeAspect": "D:/2Courses/8902/data_model_build/SlopeAspect/output_dem_slope_aspect1.nc",
    "DEM": "D:/2Courses/8902/data_model_build/DEM/DEM_bufferzone.tif",
    "NBR": "D:/2Courses/8902/data_model_build/NBR/MODIS_NBR_Nov2023.tif",
    "NDVI": "D:/2Courses/8902/data_model_build/NDVI/MODIS_NDVI_Nov2023.tif",
    "NDWI": "D:/2Courses/8902/data_model_build/NDWI/MODIS_NDWI_Nov2023.tif"
}

output_folder = "D:/2Courses/8902/reprojected_data/"

# Ensure output folder exists
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

def reproject_raster(input_path, output_path, target_crs):
    """
    Reproject a raster file (GeoTIFF) to the target CRS.
    """
    with rasterio.open(input_path) as src:
        transform, width, height = calculate_default_transform(
            src.crs, target_crs, src.width, src.height, *src.bounds)
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': target_crs,
            'transform': transform,
            'width': width,
            'height': height
        })
        with rasterio.open(output_path, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=target_crs,
                    resampling=Resampling.nearest)

def reproject_netCDF(input_path, output_path, target_crs):
    """
    Note: Reprojection for netCDF files typically involves specialized software.
    Currently, this function only copies the file for logging purposes.
    """
    shutil.copy(input_path, output_path)
    return f"Copied netCDF without reprojection: {input_path}"

def reproject_hrrr_grib(file_path, output_path):
    """
    Extract metadata from HRRR GRIB files and log CRS.
    Reprojection is not directly supported for GRIB files, but this function processes metadata.
    """
    try:
        with pygrib.open(file_path) as grib:
            msg = grib[1]  # Use the first message for metadata
            grid_type = msg.gridType
            projparams = msg.projparams if hasattr(msg, 'projparams') else "Projection parameters not available"
            return f"Processed GRIB: Grid Type: {grid_type}, Projection: {projparams}"
    except Exception as e:
        return f"Error processing GRIB file: {e}"

def process_hrrr_directory(hrrr_dir, output_dir):
    """
    Process all GRIB2 files in HRRR directory.
    """
    crs_details = []
    for root, _, files_in_dir in os.walk(hrrr_dir):
        for file in files_in_dir:
            if file.endswith(".grib2"):
                grib_file = os.path.join(root, file)
                output_file = os.path.join(output_dir, os.path.basename(file))
                result = reproject_hrrr_grib(grib_file, output_file)
                crs_details.append(f"{file}: {result}")
    return "\n".join(crs_details) if crs_details else "No GRIB2 files found in the directory"

# Main processing loop
for label, path in data_folders.items():
    print(f"Processing {label}...")
    if label == "HRRR":
        # Process HRRR files from all subdirectories
        hrrr_output_dir = os.path.join(output_folder, "HRRR")
        if not os.path.exists(hrrr_output_dir):
            os.makedirs(hrrr_output_dir)
        process_hrrr_directory(path, hrrr_output_dir)
    elif path.endswith(".tif"):  # GeoTIFF files
        output_path = os.path.join(output_folder, f"{label}_reprojected.tif")
        reproject_raster(path, output_path, TARGET_CRS)
    elif path.endswith(".nc"):  # netCDF files
        output_path = os.path.join(output_folder, f"{label}_reprojected.nc")
        reproject_netCDF(path, output_path, TARGET_CRS)
    else:
        print(f"Unsupported file format for {label}: {path}")

print(f"Reprojection complete. Files saved to {output_folder}.")
